In [1]:
import marimo as mo
import altair as alt
import polars as pl

from lancedb_tables.lance_table import LanceTable

pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

alt.data_transformers.enable("vegafusion")  # up to 100k rows

/home/evan/Documents/mev_commit_holesky_blob_analysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DataTransformerRegistry.enable('vegafusion')

In [2]:
# load data from tables
commitment_table_name: str = "commitments"
l1_tx_table_name: str = "l1_txs"
mev_boost_table_name: str = "mev_boost_blocks"
index: str = "block_number"
lance_tables = LanceTable()
uri: str = "data"  # locally saved to "data folder"

# open the mev-commit commitments table
commitments_table = lance_tables.open_table(
    uri=uri, table=commitment_table_name
)

# open the l1 txs table
l1_tx_table = lance_tables.open_table(uri=uri, table=l1_tx_table_name)
l1_tx_df = pl.from_arrow(l1_tx_table.to_lance().to_table())

# open mev-boost-blocks table
mev_boost_blocks = lance_tables.open_table(uri=uri, table=mev_boost_table_name)
mev_boost_blocks_df = pl.from_arrow((mev_boost_blocks.to_lance().to_table()))

In [3]:
holesky_mempool_df = pl.read_parquet('data/holesky_mempool.parquet').select('hash', 'event_date_time').group_by('hash').agg(pl.col('event_date_time').min()).sort(by='event_date_time')

# dataset for non blob blocks
holesky_blocks_df = pl.scan_parquet('data/holesky/blocks.parquet').rename({'number': 'block_number'})
holesky_txs_df = pl.scan_parquet('data/holesky/transactions.parquet').filter(pl.col('type') == 3)

In [4]:
# all holesky blob dataset
holesky_blocks_txs_df = (
    holesky_txs_df.join(holesky_blocks_df, on='block_number', how='left')
    .with_columns(
            (pl.col("base_fee_per_gas") * pl.col("gas_used")).alias("base_tx_fee_eth"),
            (pl.col("effective_gas_price") - pl.col("base_fee_per_gas")).alias(
                "priority_fee_gas"
            ),
            ((pl.col("max_priority_fee_per_gas") / pl.col("effective_gas_price")
            ).round(3)).alias("priority_fee_bid_percent_premium")
    )
    .with_columns(
        # have to perform priority fee calculation in this column
        (((pl.col("effective_gas_price") - pl.col("base_fee_per_gas"))
        * pl.col("gas_used")) / 10**18).alias("priority_tx_fee_eth"),
        # unit calculations for gwei and eth values
        (pl.col("base_tx_fee_eth") / 10**18).alias("base_tx_fee_eth"),
        (pl.col("priority_fee_gas") / 10**9).alias("priority_fee_gas"),
        (pl.col("base_fee_per_gas") / 10**9).alias("base_fee_per_gas"),
        pl.from_epoch('timestamp').alias('l1_block_datetime')
    )
    .collect()
    .join(holesky_mempool_df, on='hash', how='left', suffix='_l1')
    .with_columns(
        (pl.col('l1_block_datetime') - pl.col('event_date_time')).alias('mempool_time').cast(pl.Int64)
    )
    .filter(pl.col('event_date_time').is_not_null())
    )

In [5]:
# preconf dataset
commit_df = (
    pl.from_arrow(commitments_table.to_lance().to_table())
    .with_columns(
        (pl.col("dispatchTimestamp") - pl.col("decayStartTimeStamp")).alias(
            "bid_decay_latency"
        ),
        (pl.col("bid") / 10**18).alias("bid_eth"),
        pl.from_epoch("timestamp", time_unit="ms").alias("datetime"),
    )
    # bid decay calculations
    # the formula to calculate the bid decay = (decayEndTimeStamp - decayStartTimeStamp) / (dispatchTimestamp - decayEndTimeStamp). If it's a negative number, then bid would have decayed to 0
    .with_columns(
        # need to change type from uint to int to account for negative numbers
        pl.col("decayStartTimeStamp").cast(pl.Int64),
        pl.col("decayEndTimeStamp").cast(pl.Int64),
        pl.col("dispatchTimestamp").cast(pl.Int64),
    )
    .with_columns(
        (pl.col("decayEndTimeStamp") - pl.col("decayStartTimeStamp")).alias(
            "decay_range"
        ),
        (pl.col("decayEndTimeStamp") - pl.col("dispatchTimestamp")).alias(
            "dispatch_range"
        ),
    )
    .with_columns(
        (pl.col("dispatch_range") / pl.col("decay_range")).alias(
            "decay_multiplier"
        )
    )
    .with_columns(
        pl.when(pl.col("decay_multiplier") < 0)
        .then(0)
        .otherwise(pl.col("decay_multiplier"))
    )
    # calculate decayed bid. The decay multiplier is the amount that the bid decays by.
    .with_columns(
        (pl.col("decay_multiplier") * pl.col("bid_eth")).alias(
            "decayed_bid_eth"
        )
    )
    .select(
        "datetime",
        "bid_decay_latency",
        "decay_multiplier",
        "isSlash",
        "block_number",
        "blockNumber",
        "txnHash",
        "bid_eth",
        "decayed_bid_eth",
        "commiter",
        "bidder",
    )
    .rename(
        {
            "block_number": "mev_commit_block_number",
            "blockNumber": "l1_block_number",
            "txnHash": "l1_txnHash",
        }
    )
    .sort(by="datetime", descending=True)
    .join(
        l1_tx_df.rename({"hash": "l1_txnHash"}),
        on="l1_txnHash",
        how="left",
        suffix="_l1",
    ).with_columns(
        # calculate if the preconf block was the same as the l1 block the tx ended up in
        (pl.col("block_number") - pl.col("l1_block_number")).alias("l1_block_diff")
    ).filter(pl.col('type') == 3)
        .with_columns(
                (pl.col("base_fee_per_gas") * pl.col("gas_used")).alias("base_tx_fee_eth"),
                (pl.col("effective_gas_price") - pl.col("base_fee_per_gas")).alias(
                    "priority_fee_gas"
                ),
                ((pl.col("max_priority_fee_per_gas") / pl.col("effective_gas_price")
                ).round(3)).alias("priority_fee_bid_percent_premium")
            )
            .with_columns(
                # have to perform priority fee calculation in this column
                (((pl.col("effective_gas_price") - pl.col("base_fee_per_gas"))
                * pl.col("gas_used")) / 10**18).alias("priority_tx_fee_eth"),
                # unit calculations for gwei and eth values
                (pl.col("base_tx_fee_eth") / 10**18).alias("base_tx_fee_eth"),
                (pl.col("priority_fee_gas") / 10**9).alias("priority_fee_gas"),
                (pl.col("base_fee_per_gas") / 10**9).alias("base_fee_per_gas")
            )
            .with_columns(
                # calculate total fees
                (pl.col("base_tx_fee_eth") + \
                pl.col("priority_tx_fee_eth") + \
                pl.col("decayed_bid_eth")).alias("total_tx_fee_eth"),
            )
            .with_columns(
                (pl.col('priority_tx_fee_eth') / pl.col('total_tx_fee_eth')).alias('priority_fee_percent'),
                (pl.col('decayed_bid_eth') / pl.col('total_tx_fee_eth')).alias('bid_percent'),
                (pl.col('base_tx_fee_eth') / pl.col('total_tx_fee_eth')).alias('base_fee_percent')
            )
            .with_columns(
    pl.from_epoch('timestamp').dt.cast_time_unit('ms').alias('l1_block_datetime') # not strictly necessary
)
.join(holesky_mempool_df, right_on='hash', left_on='l1_txnHash', how='left', suffix='_l1')
.with_columns(
    (pl.col('l1_block_datetime') - pl.col('event_date_time')).alias('mempool_time').cast(pl.Int64),
    pl.col('bid_decay_latency').cast(pl.Int64)
)
)

## Mempool Preconf Charts

In [6]:
time_groupby_df = (
    commit_df.select('l1_block_datetime', 'l1_txnHash', 'event_date_time', 'mempool_time', 'bid_decay_latency', 'isSlash', 'l1_block_diff', 'decayed_bid_eth', 'priority_tx_fee_eth').group_by('l1_txnHash').agg(
    pl.len().alias('count'),
    pl.col('bid_decay_latency').min().alias('min_bid_decay_latency'),
    pl.col('mempool_time').min().alias('min_mempool_time'),
    pl.col('decayed_bid_eth').sum().alias('total_bid_eth'),
    pl.col('priority_tx_fee_eth').mean().alias('total_priority_tx_fee_eth'),
))

In [7]:
time_groupby_df.filter(pl.col('min_mempool_time').is_not_null()).shape

(274, 6)

In [8]:
# preconf reliability chart
alt.Chart(time_groupby_df.filter(pl.col('min_mempool_time').is_not_null())
).mark_bar().encode(
    alt.X("count:Q", bin=alt.Bin(maxbins=20), axis=alt.Axis(title='Number of Preconfs per Tx')),
    y=alt.Y('count()', axis=alt.Axis(title='Frequency')),
).properties(
    width=500,
    height=400,
    title='Preconf Relability & Resubmission Rate')

alt.Chart(...)

In [9]:
# Melt the DataFrame using Polars' melt function to make stacked bar
melted_tx_cost_df = time_groupby_df.filter(
    (pl.col('min_bid_decay_latency') < 1000) &
    (pl.col('min_mempool_time') > 0) &
    (pl.col('min_mempool_time') < 300000)
).melt(
    id_vars=[],  # No ID variables, we're melting both columns
    value_vars=['total_bid_eth', 'total_priority_tx_fee_eth'],  # Columns to melt
    variable_name='category',  # Name for the new 'category' column
    value_name='value'  # Name for the new 'value' column
)

/tmp/ipykernel_7763/2778413333.py:6: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  ).melt(


In [10]:
time_groupby_df.filter(
    (pl.col('min_bid_decay_latency') < 1000) &
    (pl.col('min_mempool_time') > 0) &
    (pl.col('min_mempool_time') < 300000)
).group_by('l1_txnHash').agg(
    pl.col('min_bid_decay_latency').mean().alias('mean_bid_decay_latency'),
    pl.col('min_mempool_time').mean().alias('mean_mempool_time'),
).select('mean_bid_decay_latency', 'mean_mempool_time').mean()

mean_bid_decay_latency,mean_mempool_time
f64,f64
75.1260162601626,97677.87804878049


In [11]:
### Alternate reliability chart
# # Create the scatterplot with stacked histogram and updated labels
# tx_fee_chart = alt.Chart(melted_tx_cost_df).mark_bar().encode(
#     x=alt.X('value:Q', bin=alt.Bin(maxbins=10), axis=alt.Axis(title='Value (ETH)')),
#     y=alt.Y('count()', axis=alt.Axis(title='Count')),
#     color=alt.Color('category:N', scale=alt.Scale(domain=['total_bid_eth', 'total_priority_tx_fee_eth'], range=['blue', 'red']), title='Category')
# ).properties(title='Fee Amount')

# preconf reliability chart
preconf_reliability = alt.Chart(time_groupby_df.filter(pl.col('min_mempool_time').is_not_null())
).mark_bar(opacity=0.4, color='red').encode(
    alt.X("count:Q", bin=alt.Bin(maxbins=20), axis=alt.Axis(title='Number of Preconfs per Tx')),
    y=alt.Y('count()', axis=alt.Axis(title='Count')),
).properties(
    width=500,
    height=400,
    title='Preconf Speed and Reliability')

# x-axis Histogram with better labeling
x_hist = alt.Chart(time_groupby_df.filter(
    (pl.col('min_bid_decay_latency') < 1000) &
    (pl.col('min_mempool_time') > 0) &
    (pl.col('min_mempool_time') < 300000) &
    (pl.col('total_bid_eth') < 0.002)
)).mark_bar().encode(
    x=alt.X('min_bid_decay_latency:Q', bin=alt.Bin(maxbins=20), axis=alt.Axis(title='Bid Latency (ms)')),
    y=alt.Y('count()', axis=alt.Axis(title='Count')),
).properties(
    height=60,
    title='Preconf Latency'
).interactive()

# y-axis Histogram with better labeling
y_hist = alt.Chart(time_groupby_df.filter(
    (pl.col('min_bid_decay_latency') < 1000) &
    (pl.col('min_mempool_time') > 0) &
    (pl.col('min_mempool_time') < 300000) &
    (pl.col('total_bid_eth') < 0.002)
)).mark_bar().encode(
    y=alt.Y('min_mempool_time:Q', bin=alt.Bin(maxbins=20), axis=alt.Axis(title='Mempool Time (ms)')),
    x=alt.X('count()', axis=alt.Axis(title='Count'))
).properties(
    width=60,
    title='Mempool Latency'
).interactive()

# Combine scatterplot with histograms and add main title
scatter_hist = (
    (preconf_reliability | y_hist)
    & x_hist
).resolve_scale(
    x='independent', 
    y='independent'
)

scatter_hist

alt.VConcatChart(...)

In [12]:
STOP

NameError: name 'STOP' is not defined

In [159]:
### CURRENT RELIABILITY CHART
# # preconf reliability chart
# preconf_reliability = alt.Chart(time_groupby_df.filter(pl.col('min_mempool_time').is_not_null())
# ).mark_bar(opacity=0.4, color='red').encode(
#     alt.X("count:Q", bin=alt.Bin(maxbins=20), axis=alt.Axis(title='Number of Preconfs per Tx')),
#     y=alt.Y('count()', axis=alt.Axis(title='Count')),
# ).properties(
#     width=500,
#     height=400,
#     title='Preconf Speed and Reliability')

# # x-axis Histogram with better labeling
# x_hist = alt.Chart(time_groupby_df.filter(
#     (pl.col('min_bid_decay_latency') < 1000) &
#     (pl.col('min_mempool_time') > 0) &
#     (pl.col('min_mempool_time') < 300000) &
#     (pl.col('total_bid_eth') < 0.002)
# )).mark_bar().encode(
#     x=alt.X('min_bid_decay_latency:Q', bin=alt.Bin(maxbins=20), axis=alt.Axis(title='Bid Latency (ms)')),
#     y=alt.Y('count()', axis=alt.Axis(title='Count')),
# ).properties(
#     height=60,
#     title='Preconf Latency'
# ).interactive()

# # y-axis Histogram with better labeling
# y_hist = alt.Chart(time_groupby_df.filter(
#     (pl.col('min_bid_decay_latency') < 1000) &
#     (pl.col('min_mempool_time') > 0) &
#     (pl.col('min_mempool_time') < 300000) &
#     (pl.col('total_bid_eth') < 0.002)
# )).mark_bar().encode(
#     y=alt.Y('min_mempool_time:Q', bin=alt.Bin(maxbins=20), axis=alt.Axis(title='Mempool Time (ms)')),
#     x=alt.X('count()', axis=alt.Axis(title='Count'))
# ).properties(
#     width=60,
#     title='Mempool Latency'
# ).interactive()

# # Combine scatterplot with histograms and add main title
# scatter_hist = (
#     (preconf_reliability | y_hist)
#     & x_hist
# ).resolve_scale(
#     x='independent', 
#     y='independent'
# )

# scatter_hist


alt.VConcatChart(...)

In [160]:
# is there a better way to size the chart?
alt.Chart(melted_tx_cost_df.filter(pl.col('value') < .002)).mark_bar().encode(
    x=alt.X('value:Q', bin=alt.Bin(maxbins=10), axis=alt.Axis(title='Value (ETH)')),
    y=alt.Y('count()', axis=alt.Axis(title='Count')),
    color=alt.Color('category:N', scale=alt.Scale(domain=['total_bid_eth', 'total_priority_tx_fee_eth'], range=['blue', 'red']), title='Category')
).properties(
    width=300,
    height=400,
    title='Fee Amount')

alt.Chart(...)

### Preconf vs Non Preconf

In [40]:
preconf_non_preconf_df = holesky_blocks_txs_df.with_columns(
    pl.when(pl.col('hash').is_in(
        commit_df['l1_txnHash'].to_list()
        )).then(True).otherwise(False).alias('preconf')
)

In [62]:
# number of blobs that settle within 2 blocks (24 seconds)
preconf_non_preconf_df.select('mempool_time').filter(pl.col('mempool_time') > 0).filter(pl.col('mempool_time') < 24_000).shape

(247116, 1)

In [63]:
# number of blobs that take longer than 2 blocks (24 seconds)
preconf_non_preconf_df.select('mempool_time').filter(pl.col('mempool_time') > 0).filter(pl.col('mempool_time') > 24_000).shape

(15684, 1)

In [158]:
alt.Chart(preconf_non_preconf_df.filter(pl.col('mempool_time') > 0).filter(pl.col('mempool_time') > 24000)
                      .filter(pl.col('mempool_time') < 300_000)
                     ).mark_bar().encode(
    x=alt.Y('mempool_time:Q', bin=alt.Bin(maxbins=30), axis=alt.Axis(title='Mempool Time (ms)')),
    y=alt.X('count()', axis=alt.Axis(title='Count')),
).properties(
    width=500,
    height=400,
    title='Blob Mempool Latency (>24 seconds in mempool)'
)


alt.Chart(...)

In [72]:
preconf_compare_df = (
    preconf_non_preconf_df.filter(pl.col('mempool_time') > 0).filter(pl.col('mempool_time') > 24_000)
    # .group_by('preconf').agg(
    # pl.col('priority_tx_fee_eth').mean().alias('mean_priority_tx_fee_eth'),
    # pl.col('mempool_time').mean().alias('mean_mempool_time'),
# )
)

In [110]:
alt.Chart(preconf_compare_df.filter(pl.col('priority_tx_fee_eth') < .0002)).mark_point().encode(
    x=alt.Y('priority_tx_fee_eth:Q', bin=alt.Bin(maxbins=30), axis=alt.Axis(title='priority fee (eth)')),
    y=alt.X('mempool_time', axis=alt.Axis(title='mempool time')),
    color='preconf'
).properties(
    width=450,
    height=300,
    title='Priority Fees (>24 seconds in mempool)'
)

alt.Chart(...)

In [161]:
priority_fee_chart = alt.Chart(preconf_compare_df.filter(pl.col('priority_tx_fee_eth') < .0002)).mark_bar().encode(
    x=alt.Y('priority_tx_fee_eth:Q', bin=alt.Bin(maxbins=30), axis=alt.Axis(title='priority fee (eth)')),
    y=alt.X('count()', axis=alt.Axis(title='Count')),
    color='preconf'
).properties(
    width=450,
    height=500,
    title='Priority Fees (>24 seconds in mempool)'
)

preconf_bid_chart = alt.Chart(time_groupby_df.filter(pl.col('min_mempool_time').is_not_null()).filter(pl.col('min_mempool_time') > 24_000).filter(pl.col('total_bid_eth') < .002)).mark_bar().encode(
    x=alt.Y('total_bid_eth', bin=alt.Bin(maxbins=30), axis=alt.Axis(title='preconf bid (eth)')),
    y=alt.X('count()', axis=alt.Axis(title='Count'))
).properties(
    width=450,
    height=500,
    title='Preconf Fees (>24 seconds in mempool)'
)

priority_fee_chart | preconf_bid_chart

alt.HConcatChart(...)

In [141]:
def calculate_base_fee_increase(initial_base_fee=3):
    block_value = initial_base_fee
    for n in range(1, 11):
        block_value *= 1.125  # Increase base fee by 12.5%
    return block_value

# Perform the calculation
result = calculate_base_fee_increase()
result

9.741963076405227

In [ ]:
STOP

### mev-boost (old)

In [4]:
def byte_to_string(hex_string):
    if hex_string == "0x":
        return ""
    # Remove the "0x" prefix and decode the hex string
    bytes_object = bytes.fromhex(hex_string[2:])
    try:
        human_readable_string = bytes_object.decode("utf-8")
    except UnicodeDecodeError:
        human_readable_string = bytes_object.decode("latin-1")
    return human_readable_string

In [5]:
mev_boost_relay_transformed_df = mev_boost_blocks_df.with_columns(
    pl.from_epoch("timestamp", time_unit="s").alias("datetime"),
    # map byte_to_string
    pl.col("extra_data")
    .map_elements(byte_to_string, return_dtype=str)
    .alias("builder_graffiti"),
    pl.when(pl.col("relay").is_null())
    .then(False)
    .otherwise(True)
    .alias("mev_boost"),
    (pl.col("value") / 10**18).round(9).alias("block_bid_eth"),
).select(
    "datetime",
    "block_number",
    "builder_graffiti",
    "mev_boost",
    "relay",
    "block_bid_eth",
    "base_fee_per_gas",
    "gas_used",
)

# transform commit_df to stsandardize to block level data
preconf_blocks_grouped_df = (
    commit_df.select(
        "l1_block_number",
        "isSlash",
        "bid_eth",
        "decayed_bid_eth",
        "commiter",
        "bidder",
    )
    .group_by("l1_block_number")
    .agg(
        pl.col("decayed_bid_eth").sum().alias("total_decayed_bid_eth"),
        pl.col("bid_eth").sum().alias("total_bid_eth"),
        pl.col("isSlash").first().alias("isSlash"),
    )
)
# join mev-boost data to preconf data
mev_boost_commit_df = mev_boost_relay_transformed_df.join(
    preconf_blocks_grouped_df,
    left_on="block_number",
    right_on="l1_block_number",
    how="left",
).with_columns(
    pl.when(pl.col("total_bid_eth").is_not_null())
    .then(True)
    .otherwise(False)
    .alias("preconf")
)

In [ ]:
# this chart shows the distribution of bid decay latencies for each bidder. 
# The distributions are different because they were sent from various places around the world, with the faster bids being closer to the provider.
alt.Chart(commit_df.filter(pl.col('bid_decay_latency') < 1000)).mark_boxplot().encode(
    alt.X('bidder', title='Bidder'),
    alt.Y('bid_decay_latency', title='Bid Decay Latency (ms)'),
    color=alt.Color("bidder", title='Bidder')
).properties(
    width=600,
    height=400,
    title='Bid Decay Latency per Bidder'
)

In [18]:
# Group by 'bidder' and calculate the summary statistics for 'bid_decay_latency'
summary_stats = commit_df.group_by('bidder').agg([
    pl.col('bid_decay_latency').quantile(0.25).alias('Q1'),
    pl.col('bid_decay_latency').median().alias('Median'),
    pl.col('bid_decay_latency').quantile(0.75).alias('Q3'),
    pl.col('bid_decay_latency').min().alias('Min'),
    pl.col('bid_decay_latency').max().alias('Max'),
    pl.len().alias('count')
])

In [ ]:
# the below table shows the summary stats for the data in the box plot. 
# Notably the addresses that sent the most bids achieved latencies as low as 13ms and 43ms, with median latencies of 119 and 71 respectively.  
summary_stats

### economic cost

In [ ]:
# the provider we tested was using a FCFS strategy to give commitments. 
# Thus the bid amount would not have any correlations with the priority fee amount for acceptance time. 
alt.Chart(
    commit_df.with_columns((pl.col('max_priority_fee_per_gas') // 10**9).alias('max_priority_fee_gwei')).filter(pl.col('max_priority_fee_gwei') < 30)
    ).mark_circle(size=60).encode(
    x='bid_decay_latency',
    y='decayed_bid_eth',
    tooltip=['decayed_bid_eth', 'max_priority_fee_gwei', 'max_fee_per_gas']
).interactive()

In [ ]:
alt.Chart(
    commit_df.with_columns((pl.col('max_priority_fee_per_gas') // 10**9).alias('max_priority_fee_gwei')).filter(pl.col('decayed_bid_eth') < .002)
).mark_bar().encode(
    alt.X('decayed_bid_eth:Q', bin=alt.Bin(maxbins=30), title='Decayed Bid (ETH)'),
    alt.Y('count()', title='Count'),
    tooltip=['count()']
).properties(
    width=600,
    height=400,
    title='Histogram of Decayed Bid (ETH)'
).interactive()

### effect on mev-boost auction

In [ ]:
mev_boost_commit_df.group_by('preconf').agg(pl.len().alias('count'))

In [57]:
curated_mev_boost_df = (
    mev_boost_commit_df.with_columns(
        (pl.col("total_decayed_bid_eth") / pl.col("block_bid_eth"))
        .round(3)
        .alias("preconf_bid_amt_pct")
    )
)

In [ ]:
alt.data_transformers.disable_max_rows()
# we basicallly randomized the preconf bid data. Then we calculated the amount of preconf bids that were collected that could be used
# to increase the size of the mev-boost bid, with the median value added to mev-boost bids being 34.5%.
# This indicates that we were overpaying with the submitted preconf bids. Ideally the value added to mev-boost bids should be as close to 0% as possible.
# Although the higher the value of the preconf, the more likely the builder is to win the auction  

alt.Chart(curated_mev_boost_df.filter(pl.col('preconf') == True)).mark_bar().encode(
    alt.X("preconf_bid_amt_pct:Q", bin=alt.Bin(maxbins=25), title='Preconfirmation Bid Amount (%)'),
    alt.Y('count()', title='# of Winning Block Bids'),
).properties(
    width=600,
    height=400,
    title='Preconf Bids Attributable to MEV-Boost Bid'
)

In [66]:
# Group by 'bidder' and calculate the summary statistics for 'bid_decay_latency'
mev_boost_summary_stats = curated_mev_boost_df.group_by('preconf').agg([
    pl.col('preconf_bid_amt_pct').quantile(0.25).alias('Q1'),
    pl.col('preconf_bid_amt_pct').median().alias('Median'),
    pl.col('preconf_bid_amt_pct').quantile(0.75).alias('Q3'),
    pl.col('preconf_bid_amt_pct').min().alias('Min'),
    pl.col('preconf_bid_amt_pct').max().alias('Max'),
    pl.len().alias('count')
])

In [ ]:
mev_boost_summary_stats

In [ ]:
curated_mev_boost_df

In [ ]:
alt.Chart(curated_mev_boost_df.filter(pl.col('mev_boost') == True).filter(pl.col('block_bid_eth') < .05)).mark_bar().encode(
    alt.X("block_bid_eth:Q", bin=alt.Bin(maxbins=50), title='bids'),
    alt.Y('count()', title='# of Winning Block Bids'),
    color=alt.Color("preconf", title='preconf')
).properties(
    width=600,
    height=400,
    title='mev-boost bids with and without preconfs'
)